In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [3]:
pwd

'/home/jovyan/capstone-52/topic_modeling_experiments/multinomialnb_svm'

In [5]:
cd ../../../capstone-52/Pickled_from_mongo/

/home/jovyan/capstone-52/Pickled_from_mongo


In [6]:
df = pd.read_pickle('../Pickled_from_mongo/combined_eg_gulf_100k_sample.p')

In [7]:
df.shape

(192936, 5)

In [8]:
df.head()

,_id,cleaned_geo,cleaned_name,cleaned_text,class
0,5a2c7a44204c9e0400cdc0e2,None,kamal,أزاي أقول لك كنا زمان والماضي كان فى الغيب بكر...,EG
1,5a2c7a44204c9e0400cdc0e3,Egypt,agabdelrehim,هي آراء آه بس أزاي أجويرو منتهي يعني أمال لو م...,EG
2,5a2c7a44204c9e0400cdc0e4,None,ElsndubadE,أنت صيني أزاي تقارن شادي بالخطيب ألي هو الوحيد...,EG
3,5a2c7a44204c9e0400cdc0e5,None,h_sawires,أزاي الناس كانت بتغرد في الخمسينات قبل إختراع ...,EG
4,5a2c7a44204c9e0400cdc0e6,Egypt,Hagerelmor,التوينز اللي معاها كل الحلو والوحش والمصايب وا...,EG


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [10]:
df_test = df['cleaned_text'].sample()

In [11]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['cleaned_text'])
X_train_counts.shape

(192936, 223045)

In [12]:
type(X_train_counts)

scipy.sparse.csr.csr_matrix

In [13]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(192936, 223045)

### MultinomialNB

In [14]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df['class'])

#### Conduct search using two different dialects and see if predicted class is accurate

In [15]:
docs_new = ['الثوره المصريه تحولت من ثورة شارع محدش يزعل', 'ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r %s' % (doc, df['class'].sample(10)))
    

'الثوره المصريه تحولت من ثورة شارع محدش يزعل' 10354    GULF
2310       EG
41816    GULF
22988    GULF
23817    GULF
43860      EG
44919    GULF
4359     GULF
17965    GULF
75680    GULF
Name: class, dtype: object
'ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح' 28081      EG
13907    GULF
77177      EG
7298     GULF
24292      EG
11671    GULF
80894      EG
79569      EG
84371      EG
9972     GULF
Name: class, dtype: object


In [16]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [17]:
text_clf.fit(df['cleaned_text'], df['class']) 

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

#### Evaluating predictive accuracy of the model.

Tagged tweets from both EG and GULF classes that have not been seen by the training data.

In [18]:
search_sentences = [
{"sentence": "الثوره المصريه تحولت من ثورة شارع محدش يزعل", "class" : "EG"},
{"sentence": "نفسي اكون زيك بعرف اطنشك أو اخليك اخر حاجة و بعد كده اضحك عليك بكلمتين و انت تصدق كل مرة عادي", "class" : "EG"},
{"sentence": "بما أن أغلب اللي متابعني مش بقدر اوصلهم أغلب الوقت. . ف كل يوم هعمل تويته آخر اليوم اللي هيعمل لايك", "class" : "EG"},
{"sentence": "مقاومتنا للأشياء طلعت بتقل مع الزمن، مبقيناش نناهد ف حاجة.. و مش عشان أحنا جامدين قوي. هو حيلنا بس", "class" : "EG"},
{"sentence": "عارف ايه احلى حاجة حاصلة ليا انى منك وانت برضه بتجرى فيا انت اخر كل يوم باخدك ف حضنى وانت اول", "class" : "EG"},
{"sentence": "القاضى اللى حكم على المعتقلين بالاعدام هو هو نفس القاضى اللى هيراقب الانتخابات", "class" : "EG"},
{"sentence": "وفجأة تيجي سيرة حاجة في وسط الكلام تقلب عليك القديم والجديد وترسم في دماغك علامات استفهام مالهاش", "class" : "EG"},
{"sentence": "السنة اللي فاتت الاعلام الانجليزي قال المفروض بيب يعرف انه في البريمييرليج لازم يتأقلم و يلعب كورتنا", "class" : "EG"},
{"sentence": "حرب و قتال و ناس تموت و هذا الدلخ يقول سعيد و مثل أجواء كرة القدم ", "class" : "GULF"},
{"sentence": "من غباء الهلالي الدلخ اللي يفتخر بفوز فريقه من قيادة رئيس الحكام كلاتنبيرغ له سنه ماسنع الحكام السعوديين", "class" : "GULF"},
{"sentence": "شفتوا هوشة شيعان وغالي لو هي بين الهلاليين كان شفتوا هاشتاق كبر راسهم المنسم وكان جاك هذا الدلخ ", "class" : "GULF"},
{"sentence": "ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح ", "class" : "GULF"},
{"sentence": "صج ياجماعه في سوال محيرني ليش المتان مافيهم النفسيه عكس الضعاف تقول خاشوقه ومنفس", "class" : "GULF"},
{"sentence": "لم نعاند التاريخ مسيو خاشوقه بل الواقع والعقلانية ابعدنا من التدمير والانفلات", "class" : "GULF"},
{"sentence": "أي والله وعندي عنه ابو خاشوقة أسرار لا تشرف قد أقولها اذا لم يلجم لسانه عن سب وطني", "class" : "GULF"},
{"sentence": "قبل ماتتكلمين يالطيبه افهمي السالفه ومنب ملزومه بسنابي اني اشرح كل شيء صارت بالتفصيل بس لانك قلق خل", "class" : "GULF"},
]

In [19]:
search_sentences_df = pd.DataFrame(search_sentences)

In [20]:
search_sentences_df.sample(4)

,class,sentence
5,EG,القاضى اللى حكم على المعتقلين بالاعدام هو هو ن...
11,GULF,ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس ما...
8,GULF,حرب و قتال و ناس تموت و هذا الدلخ يقول سعيد و ...
1,EG,نفسي اكون زيك بعرف اطنشك أو اخليك اخر حاجة و ب...


__MultinomialNB prediction score__

In [21]:
import numpy as np

docs_test = search_sentences_df['sentence']
predicted = text_clf.predict(docs_test)
np.mean(predicted == search_sentences_df['class'])  

1.0

### Support Vector Machine (SVM)

In [22]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42)),
                    ])
text_clf.fit(df['cleaned_text'], df['class'])  

predicted = text_clf.predict(docs_test)
np.mean(predicted == search_sentences_df['class']) 

0.8125

__Precision/Recall and Confusion Matrix for SVM__

In [23]:
from sklearn import metrics
print(metrics.classification_report(search_sentences_df['class'], predicted, 
                                    target_names=['EG',"GULF"]))

             precision    recall  f1-score   support

         EG       1.00      0.62      0.77         8
       GULF       0.73      1.00      0.84         8

avg / total       0.86      0.81      0.81        16



In [24]:
metrics.confusion_matrix(search_sentences_df['class'], predicted)

array([[5, 3],
       [0, 8]])

In [ ]:
score(X, y[, sample_weight])

__Parameter tuning using grid search__

In [25]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }

In [26]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [27]:
gs_clf = gs_clf.fit(df['cleaned_text'], df['class'])

__Using as a classifier to predict dialect__

In [28]:
search_sentences_df['class'][gs_clf.predict(['الثوره المصريه تحولت من ثورة شارع محدش يزعل'])]

GULF    NaN
Name: class, dtype: object

In [29]:
gs_clf.best_score_  

0.74826885599369741

In [30]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [31]:
cv_results_df = pd.DataFrame(gs_clf.cv_results_)

In [34]:
high_mean_test_mask = cv_results_df[cv_results_df['mean_test_score'] >= 0.70]

In [35]:
high_mean_test_mask

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf__alpha,param_tfidf__use_idf,param_vect__ngram_range,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
4,2.869609,1.051105,0.728096,0.742352,0.001,True,"(1, 1)","{'clf__alpha': 0.001, 'tfidf__use_idf': True, ...",3,0.735590,0.740948,0.749487,0.747909,0.699212,0.738200,0.060067,0.054278,0.021198,0.004086
5,9.780769,2.322877,0.748269,0.754986,0.001,True,"(1, 2)","{'clf__alpha': 0.001, 'tfidf__use_idf': True, ...",1,0.765226,0.767724,0.761755,0.761763,0.717824,0.735471,0.344075,0.116200,0.021574,0.014012
6,2.787026,1.026807,0.736882,0.755953,0.001,False,"(1, 1)","{'clf__alpha': 0.001, 'tfidf__use_idf': False,...",2,0.746303,0.752579,0.753701,0.757604,0.710640,0.757675,0.057421,0.063561,0.018799,0.002386
7,9.119327,2.148480,0.724090,0.734339,0.001,False,"(1, 2)","{'clf__alpha': 0.001, 'tfidf__use_idf': False,...",4,0.734097,0.731836,0.738976,0.738136,0.699196,0.733046,0.290211,0.164013,0.017715,0.002730
